In [58]:
import dowhy
from dowhy import CausalModel
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from IPython.core.display import Image, display
import dowhy.causal_estimators.linear_regression_estimator

import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
from sys import path
import os

for dirname, _, filenames in os.walk("../../../resources/"):
    for filename in filenames:
        print(os.path.join(dirname, filename))
path.append(os.path.realpath("../../../"))

from custom import functions

warnings.filterwarnings("ignore")

../../../resources/allColProSol.xlsx
../../../resources/dataset_a_2021.csv
../../../resources/dataset_a_2021c2.csv
../../../resources/dataset_a_2021v1.xlsx
../../../resources/dataset_unab_ORIGINAL SIN P1.csv
../../../resources/dataset_unab_P1.csv
../../../resources/exitoFallidoEnviosProgramaSol.xlsx
../../../resources/exitoFallidoProgramaSol.xlsx
../../../resources/exitosoFallidosEnviosAllColProSol.xlsx
../../../resources/exitosoFallidosEnviosAllColSol.xlsx
../../../resources/exitososFallidosEnviosSol.xlsx
../../../resources/Hito12sinColCeroExitosoFallidosSol.xlsx
../../../resources/sinColCeroExitosoFallidosEnviosSol.xlsx
../../../resources/sinColCeroExitosoFallidosSol.xlsx
../../../resources/v2_hitosExitoFalloColESol1.csv
../../../resources/v2_hitosExitoFalloColESol1Prograna.csv
../../../resources/v2_hitosExitoFalloSol1Programa.csv
../../../resources/causalidad\causalidad.dot
../../../resources/causalidad\causalidad.png
../../../resources/causalidad\causalidad_e29.dot
../../../resourc

In [59]:
# Read and preview data
df = pd.read_csv(
    "../../../resources/v2_hitosExitoFalloColESol1.csv",
    delimiter=";",
    skipinitialspace=True,
)

In [60]:
# creando columna aprobado y con la funcion set_in_aprobado_nota poblamos la nueva columna.
df["aprobado"] = df.apply(lambda x: functions.set_in_aprobado_nota(x["sol1"]), axis=1)
# revisamos la existencia de la nueva columna.
print(df.columns)

Index(['hito1', 'hito2', 'exitosos', 'fallidos', 'e0', 'e1', 'e2', 'e3', 'e4',
       'e5', 'e6', 'e7', 'e8', 'e9', 'e10', 'e11', 'e12', 'e13', 'e14', 'e15',
       'e16', 'e17', 'e18', 'e19', 'e20', 'e21', 'e22', 'e23', 'e24', 'e25',
       'e26', 'e27', 'e28', 'e29', 'e30', 'e31', 'e32', 'e33', 'e34', 'e35',
       'e36', 'e37', 'e38', 'e39', 'e40', 'e41', 'e42', 'e43', 'e44', 'e45',
       'e46', 'e47', 'e48', 'e49', 'e50', 'e51', 'e52', 'sol1', 'aprobado'],
      dtype='object')


In [61]:
# Paso 1: Modelar un problema causal
model_hito1 = CausalModel(
    data=df,
    treatment="hito1",  # Variable tratada (exposición)
    outcome="aprobado",  # Variable de resultado
    common_causes=[
        "exitosos",
        "fallidos",
        "e42",
        "e35",
        "e32",
        "e29",
        "e26",
        "e25",
        "e23",
        "e22",
        "e18",
        "e17",
        "e13",
        "e10",
        "e8",
        "e7",
        "e4",
        "e3",
        "e0",
    ],  # Variables de causa común
)
# Visualizar el modelo causal con la variable de causa común no observada
# model_hito1.view_model(
#     layout="dot",
#     file_name="../../../resources/causalidad/graph_causal_model_hito1_pruebaa",
# )

In [62]:
# Paso 1: Modelar un problema causal
model_exitosos = CausalModel(
    data=df,
    treatment="exitosos",  # Variable tratada (exposición)
    outcome="aprobado",  # Variable de resultado
    common_causes=[
        "hito1",
        "fallidos",
        "e42",
        "e35",
        "e32",
        "e29",
        "e26",
        "e25",
        "e23",
        "e22",
        "e18",
        "e17",
        "e13",
        "e10",
        "e8",
        "e7",
        "e4",
        "e3",
        "e0",
    ],  # Variables de causa común
)
# Visualizar el modelo causal con la variable de causa común no observada
# model_exitosos.view_model(
#     layout="dot",
#     file_name="../../../resources/causalidad/graph_causal_model_exitosos_pruebaa",
# )

In [63]:
# Paso 1: Modelar un problema causal
model_fallidos = CausalModel(
    data=df,
    treatment="fallidos",  # Variable tratada (exposición)
    outcome="aprobado",  # Variable de resultado
    common_causes=[
        "hito1",
        "exitosos",
        "e42",
        "e35",
        "e32",
        "e29",
        "e26",
        "e25",
        "e23",
        "e22",
        "e18",
        "e17",
        "e13",
        "e10",
        "e8",
        "e7",
        "e4",
        "e3",
        "e0",
    ],  # Variables de causa común
)
# Visualizar el modelo causal con la variable de causa común no observada
# model_fallidos.view_model(
#     layout="dot",
#     file_name="../../../resources/causalidad/graph_causal_model_fallidos_pruebaa",
# )

In [64]:
modelos = [model_hito1, model_exitosos, model_fallidos]
resultados = []  # Lista para almacenar los resultados de cada modelo

In [65]:
for i, model in enumerate(modelos):
    print(f"Procesando modelo {i + 1} de {len(modelos)}")

    # Paso 2: Identificar el estimando objetivo bajo el modelo
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)

    # Paso 3: Estimar el efecto causal usando un método basado en árboles
    estimate = model.estimate_effect(
        identified_estimand,
        method_name="backdoor.econml.dml.DML",
        control_value=0,
        treatment_value=1,
        target_units="ate",  # Average Treatment Effect
        method_params={
            "init_params": {
                "model_y": RandomForestRegressor(),
                "model_t": RandomForestRegressor(),
                "model_final": RandomForestRegressor(
                    max_depth=10,
                    min_samples_split=10,
                    min_samples_leaf=5,
                    random_state=1502,
                    n_estimators=500,
                ),
                "featurizer": None,
            },
            "fit_params": {},
        },
    )

    # Refutador de datos aleatorios
    refute1 = model.refute_estimate(
        identified_estimand, estimate, method_name="random_common_cause"
    )
    # Refutador de causa común no observada
    refute2 = model.refute_estimate(
        identified_estimand,
        estimate,
        method_name="add_unobserved_common_cause",
        confounders_effect_on_treatment="binary_flip",
        confounders_effect_on_outcome="binary_flip",
        effect_strength_on_treatment=0.01,
        effect_strength_on_outcome=0.02,
    )
    # Paso 4: Refutar el estimado obtenido
    refute3 = model.refute_estimate(
        identified_estimand,
        estimate,
        method_name="placebo_treatment_refuter",
        placebo_type="permute",
    )

    # Almacenar los resultados en la lista de resultados
    resultados.append(
        {
            "modelo": f"modelo_{i + 1}",
            "Estimación del Efecto Causal": estimate.value,
            "Tipo (Causa Común Aleatoria)": refute1.refutation_type,
            "CCA Efecto Estimado": refute1.estimated_effect,
            "CCA - Nuevo Efecto": refute1.new_effect,
            "CCA - Valor p": refute1.refutation_result["p_value"],
            "Tipo (Causa Común No Observada)": refute2.refutation_type,
            "CCNO - Efecto Estimado": refute2.estimated_effect,
            "CCNO - Nuevo Efecto": refute2.new_effect,
            "Tipo (Tratamiento Placebo)": refute3.refutation_type,
            "TP - Efecto Estimado": refute3.estimated_effect,
            "TP - Nuevo Efecto": refute3.new_effect,
            "TP - Valor p": refute3.refutation_result["p_value"],
        }
    )

Procesando modelo 1 de 3


KeyboardInterrupt: 

In [ ]:
# Convertir la lista de resultados en un DataFrame de Pandas para una fácil manipulación y visualización
results_df = pd.DataFrame(resultados)

In [ ]:
# Configurar el tamaño del gráfico
plt.figure(figsize=(15, 10))

# Crear un rango de valores x para cada variable de tratamiento
x = np.arange(len(results_df["Variable de Tratamiento"]))

# Ancho de las barras
width = 0.1

# Crear gráficos de barras para cada métrica
plt.bar(
    x - 2 * width,
    results_df["Estimación del Efecto Causal"],
    width,
    label="Estimación del Efecto Causal",
)
plt.bar(x - width, results_df["CCA - Nuevo Efecto"], width, label="CCA - Nuevo Efecto")
plt.bar(x, results_df["CCNO - Nuevo Efecto"], width, label="CCNO - Nuevo Efecto")
plt.bar(x + width, results_df["TP - Nuevo Efecto"], width, label="TP - Nuevo Efecto")

# Añadir etiquetas, título, leyenda, etc.
plt.ylabel("Valor")
plt.title("Resultados del análisis causal por variable de tratamiento aprobado")
plt.xticks(x, results_df["Variable de Tratamiento"])
plt.legend()

# Mostrar el gráfico
plt.tight_layout()
plt.show()

In [ ]:
print(results_df)